In [2]:
### SIMULATED_BNS_RATES.IPYNB -- calculate expected astrophysical rates within sensitive volume, given total astrophysical rate estimate

In [3]:
# load packages

import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# import

#! python3 -m pip install bilby # only need to run this once
#! python3 -m pip install lalsuite # only need to run this once

#import bilby
#import lal
#import lalsimulation as lalsim
import astropy.units as u
from astropy.coordinates import Distance
from astropy.cosmology import Planck15 as cosmo

In [5]:
# user input -- realistic rate, no sfr scenario

BNS_RATE = 440. # Gpc^-3 yr^-1, cf. rates in https://arxiv.org/abs/2111.03634

ZMIN = 1e-6
DLMIN = Distance(z=ZMIN, unit=u.Mpc).value
DLMAX_DICT = {'O4': 190., 'O5': 330., '3G': 3000., 'z10': 100000.} # Mpc; 200, 350, 100000 (z=9.5) are reasonable range estimates for O4, O5, 3G; see https://arxiv.org/abs/1304.0670, https://arxiv.org/abs/2109.09882

def SFR(z): # star formation rate as a function of redshift
    
    if np.isscalar(z): z = np.array([z])
    else: z = np.array(z)
    
    return 1.

def DL_PDF(dl,params): # luminosity distance distribution probability density function

  dlmin, dlmax = params
	
  if np.isscalar(dl): dl = np.array([dl])
  else: dl = np.array(dl)
  Z = np.zeros(len(dl))
	
  z = np.array([Distance(d,unit=u.Mpc).compute_z(cosmology=cosmo) for d in dl])
  p = 4.*np.pi*dl**2*SFR(z)/(cosmo.H(z).value*(1.+z)**3) # uniform in comoving volume distribution, with Madau-Dickinson SFR; see https://arxiv.org/abs/1505.05607, https://arxiv.org/abs/1805.10270
	
  return np.where((dl > dlmax) | (dl < dlmin), Z, p) # this enforces the dlmin and dlmax cutoffs

In [6]:
z_grid = np.linspace(ZMIN,Distance(1.,unit=u.Mpc).compute_z(cosmology=cosmo),1000)
norm = 1e9*np.trapz(DL_PDF(Distance(z=z_grid, unit=u.Mpc).value,(DLMIN,DLMAX_DICT['z10'])),z_grid)
print(norm) # this normalization ensures local rate (out to 1 Mpc) matches BNS_RATE, 1e9 converts Gpc^-3 to Mpc^-3

13993.490118213776


In [8]:
z_grid = np.linspace(ZMIN,Distance(DLMAX_DICT['O4'],unit=u.Mpc).compute_z(cosmology=cosmo),1000)
BNS_RATE*np.trapz(DL_PDF(Distance(z=z_grid, unit=u.Mpc).value,(DLMIN,DLMAX_DICT['z10'])),z_grid)/norm

<Quantity 2.59156526>

In [9]:
z_grid = np.linspace(ZMIN,Distance(DLMAX_DICT['O5'],unit=u.Mpc).compute_z(cosmology=cosmo),1000)
BNS_RATE*np.trapz(DL_PDF(Distance(z=z_grid, unit=u.Mpc).value,(DLMIN,DLMAX_DICT['z10'])),z_grid)/norm

<Quantity 12.23900805>

In [10]:
z_grid = np.linspace(ZMIN,Distance(DLMAX_DICT['3G'],unit=u.Mpc).compute_z(cosmology=cosmo),1000)
BNS_RATE*np.trapz(DL_PDF(Distance(z=z_grid, unit=u.Mpc).value,(DLMIN,DLMAX_DICT['z10'])),z_grid)/norm

<Quantity 2519.43585616>

In [11]:
# user input -- realistic rate, sfr scenario

BNS_RATE = 440. # Gpc^-3 yr^-1, cf. rates in https://arxiv.org/abs/2111.03634

ZMIN = 1e-6
DLMIN = Distance(z=ZMIN, unit=u.Mpc).value
DLMAX_DICT = {'O4': 190., 'O5': 330., '3G': 3000., 'z10': 100000.} # Mpc; 200, 350, 100000 (z=9.5) are reasonable range estimates for O4, O5, 3G; see https://arxiv.org/abs/1304.0670, https://arxiv.org/abs/2109.09882

def SFR(z): # star formation rate as a function of redshift
    
    if np.isscalar(z): z = np.array([z])
    else: z = np.array(z)
    
    return  (1.+1./2.9**5.6)*(1.+z)**2.7/(1.+((1.+z)/2.9)**5.6) # Madau-Dickinson SFR, up to normalization

def DL_PDF(dl,params): # luminosity distance distribution probability density function

  dlmin, dlmax = params
	
  if np.isscalar(dl): dl = np.array([dl])
  else: dl = np.array(dl)
  Z = np.zeros(len(dl))
	
  z = np.array([Distance(d,unit=u.Mpc).compute_z(cosmology=cosmo) for d in dl])
  p = 4.*np.pi*dl**2*SFR(z)/(cosmo.H(z).value*(1.+z)**3) # uniform in comoving volume distribution, with Madau-Dickinson SFR; see https://arxiv.org/abs/1505.05607, https://arxiv.org/abs/1805.10270
	
  return np.where((dl > dlmax) | (dl < dlmin), Z, p) # this enforces the dlmin and dlmax cutoffs

In [12]:
z_grid = np.linspace(ZMIN,Distance(1.,unit=u.Mpc).compute_z(cosmology=cosmo),1000)
norm = 1e9*np.trapz(DL_PDF(Distance(z=z_grid, unit=u.Mpc).value,(DLMIN,DLMAX_DICT['z10'])),z_grid)
print(norm) # this normalization ensures local rate (out to 1 Mpc) matches BNS_RATE, 1e9 converts Gpc^-3 to Mpc^-3

13999.866149428628


In [13]:
z_grid = np.linspace(ZMIN,Distance(DLMAX_DICT['O4'],unit=u.Mpc).compute_z(cosmology=cosmo),1000)
BNS_RATE*np.trapz(DL_PDF(Distance(z=z_grid, unit=u.Mpc).value,(DLMIN,DLMAX_DICT['z10'])),z_grid)/norm

<Quantity 2.81290043>

In [14]:
z_grid = np.linspace(ZMIN,Distance(DLMAX_DICT['O5'],unit=u.Mpc).compute_z(cosmology=cosmo),1000)
BNS_RATE*np.trapz(DL_PDF(Distance(z=z_grid, unit=u.Mpc).value,(DLMIN,DLMAX_DICT['z10'])),z_grid)/norm

<Quantity 14.04992793>

In [15]:
z_grid = np.linspace(ZMIN,Distance(DLMAX_DICT['3G'],unit=u.Mpc).compute_z(cosmology=cosmo),1000)
BNS_RATE*np.trapz(DL_PDF(Distance(z=z_grid, unit=u.Mpc).value,(DLMIN,DLMAX_DICT['z10'])),z_grid)/norm

<Quantity 5822.02797352>